<a href="https://colab.research.google.com/github/danielbauer1979/CAS_PredMod/blob/main/pa_pynb_sess5_NonLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Non-linear Regression Techniques and Generalized Additive Models

Daniel Bauer, 2022

In this tutorial, we will get acquainted with non-linear models.  In particular, we introduce a number of non-linear regression techniques -- including polynomial regression, regression splines, smoothing splines, and local regression -- in a simple example setting.  The second part focuses on Generalized Linear Models -- or GAMs.

Some of the techniques we introduce don't exist in the packages that come with Colab. So we will install two additional libraries: scikit-lego for local regression and pygam for GAMs.

In [ ]:
!pip install scikit-lego
!pip install pygam

With that, let's install the relevat libraries.

In [2]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import SplineTransformer
from sklearn.pipeline import Pipeline

from sklego.linear_model import LowessRegression

from patsy import dmatrix
from pygam import LinearGAM, LogisticGAM, GAM, s, f, l

import statsmodels.api as sm

And clone our git repository so as to have access to the data.

In [ ]:
!git clone https://github.com/danielbauer1979/CAS_PredMod.git

## Non-linear Regression Models

### Primer on Non-linear Regression Techniques

Non-linear models expand on the more foundational models that we discussed before, particularly linear regression.  We consider three different though related categories of approaches, which are relatively straightforward in the context of a single feature, i.e. if there is only one $x$.

1. Use *transformations* of $x$: A traditional approach that falls in this category is *polynomial regression*, where we simply add powers of the feature $x$ -- i.e. $x^2$, $x^3$, etc. -- to the regression problem.  However, polynomials have some limitations, particularly in extremal areas, because they fit the regression function globally.  Instead, in *spline regression*, piecewise polynomials that are only fit between *knots* are used, but one imposes restrictions such that the fit is still continuous and smooth.  With so-called *natural splines*, a linear function is used in the extremal (corner) areas so as to avoid erratic behavior when extrapolating.  Depending on the number of knots, the function can mimic more or less arbitrary shapes.

2. Using an arbitrary function but *penalizing* said function for variation.  This yields a so-called *smoothing spline* (so the approach is related to 1.), but it also depends on a smoothing parameters that governs how much variation is allowed.  Similar to regularized methods, this parameter can be used to control *model complexity*.

3. *Local regression*: Instead of using all points for predicting at a given point $x_0$, we run a *local* regression where we put more weight on the data points that are close to $x_0$ and less weight on data points that are far away.  The weighting is typically done via a so-called *kernel* function (generalized bell curve) so this is also referred to as *kernel regression*.


### Example in the context of height-weight relationship

We use a straightforward example: Regressing the weight of people on their heights. The relevant dataset is taken from a [well known example dataset from Davis (1990)](https://rdrr.io/cran/carData/man/Davis.html):

In [12]:
hwdata2 = pd.read_csv('CAS_PredMod/pa_data_davis.csv', index_col=0) 
hwdata = hwdata2.sort_values('height')

Let's take a look:

In [ ]:
hwdata2.head()

And let's defined the $x$ and $y$ vectors:

In [14]:
X = hwdata2[['height']]
y = hwdata2['weight']

Let's start with polynomial regression, where we are taking advantage of so-called *model pipelines* within scikit-learn, where we "pipe" the regression model into our linear regression model:

In [22]:
polynomial_regression = Pipeline([('poly', PolynomialFeatures(degree=4)),('linear', LinearRegression(fit_intercept=False))])
polynomial_regression = polynomial_regression.fit(X, y)
polynomial_regression.named_steps['linear'].coef_

array([-4.57931469e+04,  1.10301542e+03, -9.93090159e+00,  3.96257277e-02,
       -5.90686627e-05])

Let's visualize via a scatter plot with polynomial regression line:

In [ ]:
height_grid = np.arange(hwdata.height.min(), hwdata.height.max()).reshape(-1,1)
pred_poly = polynomial_regression.predict(hwdata[['height']])
plt.figure(figsize=(15,6))
plt.scatter(hwdata.height, hwdata.weight, facecolor='None', edgecolor='k', alpha=0.3)
plt.plot(hwdata.height, pred_poly, color='g', label='polynomial regression df=4')

It seems like we may be overfitting a bit...

Let's look at regression splines, again using a model pipeline (we get splines via 'SplineTransformer'):

In [ ]:
spline_regression = Pipeline([('splines', SplineTransformer(degree=2, n_knots=3, extrapolation='linear')),('linear', LinearRegression(fit_intercept=False))])
spline_regression = spline_regression.fit(X, y)
spline_regression.named_steps['linear'].coef_

Again, let's visualize via a scatter plot with a spline regression line:

In [ ]:
height_grid = np.arange(hwdata.height.min(), hwdata.height.max()).reshape(-1,1)
pred_splines = spline_regression.predict(hwdata[['height']])
plt.figure(figsize=(15,6))
plt.scatter(hwdata.height, hwdata.weight, facecolor='None', edgecolor='k', alpha=0.3)
plt.plot(hwdata.height, pred_splines, color='g', label='Spline regression')

Looks neat.

Finally, let's run a local regression, where we rely on 'LowessRegression' from scikit-lego:

In [19]:
local_regression = LowessRegression(sigma=50).fit(X,y)

Again, let's visualize via a scatter plot with a regression line:

In [ ]:
height_grid = np.arange(hwdata.height.min(), hwdata.height.max()).reshape(-1,1)
pred_local = local_regression.predict(hwdata[['height']])
plt.figure(figsize=(15,6))
plt.scatter(hwdata.height, hwdata.weight, facecolor='None', edgecolor='k', alpha=0.3)
plt.plot(hwdata.height, pred_local, color='g', label='local regression')

Looks neat.

So the moral of the story is that these techniques present very neat tools for modeling non-linear relationships. 

Let's summarize by plotting altogether:

In [ ]:
plt.figure(figsize=(15,6))
plt.scatter(hwdata.height, hwdata.weight, facecolor='None', edgecolor='k', alpha=0.3)
plt.plot(hwdata.height, pred_poly, color='r', label='polynomial regression df=4')
plt.plot(hwdata.height, pred_splines, color='b', label='Natural spline df=4')
plt.plot(hwdata.height, pred_local, color='y', label='Local regression')

## Generalized Additive Models

As depicted, non-linear regression models are powerful approaches for depicting non-linear relationships -- with the key caveat that our explanatory variable had a single dimension only.  

One learner -- so-called **Generalized Additive Models** (GAMs) -- leverages the performance of non-linear regression models in lower dimensions but imposes an *additive* structure between the functions of the individual features:
$$
g\left(\mathbb{E}\left[Y | X_1,...,X_p\right]\right) = \alpha + f_1(X_1) + \ldots + f_p(X_p).
$$
Hence, in case of a basic regression problem where $g(x)=x$ we have the special case of *Additive Models*:
$$
Y = \alpha + f_1(X_1) + \ldots + f_p(X_p) +\varepsilon.
$$
As such, GAMs take on an intermediate position between linear regression and a general non-linear model:  They generalize the impact each feature can have on the outcome, but they keep the same structure on their relationship.  In particular, they do not allow for rich interactions between the variables -- which is the key downside of GAMs.

###Wage Regression Example

To see how GAMs work in a relevant example, let us consider the wage regression example from the texbook [ISLR](https://www.statlearning.com/).  Here, we model wages as a function of the year, age, and education level:
$$
\text{wage}_i = \alpha + f_1(\text{year}) + f_2(\text{age}) + \text{education level}_i + \varepsilon
$$

In [4]:
mydata = pd.read_csv('CAS_PredMod/pa_data_wage.csv', index_col=0)

We use the Pygam package. Python requires mannually dummifying categorical variables for this algorithm.

In [25]:
X_ = mydata[['year','age']]
dummies = pd.get_dummies(mydata['education'], drop_first=True)
X = pd.concat([X_, dummies], axis = 1)
y = mydata['wage']

Now we need to specify the model, deciding whether we want to treat the different variables as spline (s), factor (f), linear (l), etc)

In [ ]:
gam = LinearGAM(s(0,n_splines=6)+s(1,n_splines=7)+f(2)+f(3)+f(4)+f(5)).fit(X, y)
gam.summary()

Let's visualize, where remember we dummified the "education" variable into four indicators:

In [ ]:
plt.rcParams['figure.figsize'] = (28, 8)
fig, axs = plt.subplots(1, 6)
titles = list(X.columns)
for i, ax in enumerate(axs):
  XX = gam.generate_X_grid(term=i)
  ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
  ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1],c='k', ls='--')
plt.show()

We can get presictions as:

In [28]:
preds = gam.predict(X)

One neat aspect of this packages is that we are considering **G**AMs, i.e. we can define alternate link functions or distributions. Let's do a logistic GAM, where our $y$ variable is a one/zero variable with an indicator of whether the wage is above \$250.

In [29]:
X = mydata[['year','age']]
y = mydata['wage'] > 250

Here we can control the hyper-parameters n_splines for splines and lambdas for the penalty.

In [ ]:
gam5 = LogisticGAM(l(0)+s(1, n_splines=50), lam=0.6).fit(X, y.values.astype(int))
gam5.summary()

Let's visualize the results:

In [ ]:
plt.rcParams['figure.figsize'] = (28, 8)
fig, axs = plt.subplots(1, 2)
titles = list(X.columns)
for i, ax in enumerate(axs):
  XX = gam5.generate_X_grid(term=i)
  ax.plot(XX[:, i], gam5.partial_dependence(term=i, X=XX), c='g')
  ax.plot(XX[:, i], gam5.partial_dependence(term=i, X=XX, width=.95)[1],c='k', ls='--')
ax.set_title(titles[i]);
plt.show()